In [20]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

dataset = np.array([[2.771244718,1.784783929,0],[1.728571309,1.169761413,0],
           [3.678319846,2.81281357,0],[3.961043357,2.61995032,0],
           [2.999208922,2.209014212,0],[7.497545867,3.162953546,1],
           [9.00220326,3.339047188,1],[7.444542326,0.476683375,1],
           [10.12493903,3.234550982,1],[6.642287351,3.319983761,1]])
print(dataset[:,0])
print(list(dataset[:,-1]).count(0))
values_label = set(dataset[:,-1])
print(values_label)

[  2.77124472   1.72857131   3.67831985   3.96104336   2.99920892
   7.49754587   9.00220326   7.44454233  10.12493903   6.64228735]
5
{0.0, 1.0}


In [252]:


def split_data(index, value, datas):
    left = datas[datas[:,index] < value]
    right = datas[datas[:,index] >= value]
    return left,right

#计算基尼系数
def get_gini_index(splits):
    gini_weighted = 0.0
    for split in splits:
        if len(split) > 0:
            precent = len(split) / m
            #print("precent:",precent)
            cnt = list(split[:,-1]).count(value)
            split_precent = cnt/len(split)
            #print("split precent:",split_precent)
            gini_split = split_precent ** 2 + (1-split_precent) ** 2
            #print("gini splt:",gini_split)
            gini_weighted += precent * gini_split
    return gini_weighted

#返回最优切分位置及阈值
def get_split(datas):
    m = datas.shape[0]
    n = datas.shape[1]
    gini_max = 0.
    for i in range(n-1):
        for j in range(m):
            value = datas[j][i]
            splits = split_data(i, value, datas)
            gini = get_gini_index(splits)
            #print('X%d < %.3f Gini=%.3f' % ((i+1), value, gini))
            if gini > gini_max:
                gini_max = gini
                b_col,b_index,b_value, b_gini, b_split = i, j, value,gini_max, splits
    return {'col':b_col,'index':b_index, 'value':b_value,'gini':b_gini, 'splits':b_split}

#输出叶节点的类别
def to_terminal(split):
    outcomes = [row[-1] for row in split]
    return max(set(outcomes), key=outcomes.count)        

#构造左右子树
def split_node(node,max_depth,min_size,depth):
    #得到高度为2的决策树
    child_left, child_right = node['splits']
    del node['splits']
    #左右分支有一个为空
    #print(type(child_left), type(child_right))
    if len(child_left) == 0 or len(child_right) == 0:
        node['left'] = node['right'] = to_terminal(list(child_left) + list(child_right))
        return 
    #检查当前高度是否超出max_depth
    if depth >= max_depth:
        print('haha')
        node['left'], node['right'] = to_terminal(child_left),to_terminal(child_right)
        return
    #检查左右子树的数据个数是否允许进一步划分
    if len(child_left) <= min_size:
        node['left'] = to_terminal(child_left)
    else:
        node['left'] = get_split(child_left)
        split_node(node['left'],max_depth,min_size,depth+1)
        
    if len(child_right) <= min_size:
        node['right'] = to_terminal(child_right)
    else:
        node['right'] = get_split(child_right)
        split_node(node['right'],max_depth,min_size,depth+1)

#构建树
def build_tree(datas,max_depth,min_size,depth):
    datas = np.array(datas)
    root = get_split(datas)
    split_node(root,max_depth,min_size,depth)
    return root

#输出决策树的规则
def print_tree(node, depth=0):
    if isinstance(node, dict):
        print("%s[X%d < %.3f]" % (depth *' ', node['col']+1, node['value']))
        print_tree(node['left'],  depth + 1)
        print_tree(node['right'], depth + 1)
    else:
        print('%s[%s]' % ((depth*' ', node)))

        
        
#预测单个数据
def judge(node, x):
    if x[node['col']] <  node['value']:
        if isinstance(node['left'], dict):
            return judge(node['left'],x)
        else:
            label_pred = node['left']
            return label_pred
    else:
        if isinstance(node['right'], dict):
            # {'left': 1.0, 'col': 2, 'value': -4.3838999999999997, 'right': 0.0, 'index': 171, 'gini': 49.394446270128164}
            return judge(node['right'],x)
        else:
            label_pred = node['right']
            return label_pred       

#预测数据集   
def predict(node,datas):
    y_predict = []
    for data in datas:
        label = judge(node,data) 
        y_predict.append(label)
    return y_predict
        
#交叉验证
def cross_validation_splt(dataset, n_folds):
    dataset_copy = list(dataset)
    fold_size = int(len(dataset_copy) / n_folds) + 1
    
    dataset_split = []
    for i in range(n_folds):
        fold = []
        while len(fold) < fold_size and len(dataset_copy) > 0:
            index = np.random.randint(len(dataset_copy),size=1)
            fold.append(dataset_copy.pop(index[0]))
        dataset_split.append(fold)
    return dataset_split

#计算预测正确率
def accuracy_metric(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0

#留一验证评估算法
def evaluate_algorithm(dataset, n_folds):
    folds =  cross_validation_splt(datas,n_folds)
    scores = []
    for i, fold in enumerate(folds):
        if 1:
            print("第%d次" % (i))
            train_set = folds.copy()
            del train_set[i]
            train_set = sum(train_set, [])
                #print("i=",i,"train_set:",len(train_set))
            test_set = fold
                #print("i=",i,"test_set:",len(test_set))
            root = build_tree(train_set,2,10,1)
            print(root)
            y_predict = predict(root, test_set)
            print("predict前五行:",y_predict[0:5],"predict大小：",len(y_predict))
            y_true = np.array(test_set)[:,-1]
            score = accuracy_metric(y_true, y_predict)
            scores.append(score)
            print("add score")
    return scores
        
        

print('done!!!!')
# m = dataset.shape[0] 
# n = dataset.shape[1] 
# # print(get_split(dataset))
# tree = build_tree(dataset,1,1,1)
# print(tree)
# y_pred = predict(tree,dataset)
# print(y_pred)
# print(dataset[:,-1])

            
    

done!!!!


In [254]:
datas = np.loadtxt("datas\data_banknote_authentication.csv", delimiter=",")        
print(datas.shape)
print(evaluate_algorithm(datas,5))
print("good job!!")

(1372, 5)
第0次
haha
haha
{'left': {'left': 1.0, 'col': 1, 'value': 6.8741000000000003, 'right': 0.0, 'index': 114, 'gini': 45.72279236833512}, 'col': 0, 'value': 0.32229999999999998, 'right': {'left': 1.0, 'col': 2, 'value': -4.3838999999999997, 'right': 0.0, 'index': 345, 'gini': 48.92868801004395}, 'index': 780, 'gini': 82.47225181372646}
predict前五行: [1.0, 0.0, 0.0, 0.0, 1.0] predict大小： 275
add score
第1次
haha
haha
{'left': {'left': 1.0, 'col': 1, 'value': 9.5663, 'right': 0.0, 'index': 21, 'gini': 41.23381642512077}, 'col': 0, 'value': -0.29509999999999997, 'right': {'left': 0.0, 'col': 0, 'value': 1.5940000000000001, 'right': 0.0, 'index': 373, 'gini': 51.75814891438363}, 'index': 573, 'gini': 82.2419146210289}
predict前五行: [1.0, 0.0, 1.0, 1.0, 1.0] predict大小： 275
add score
第2次
haha
haha
{'left': {'left': 1.0, 'col': 1, 'value': 5.8974000000000002, 'right': 0.0, 'index': 283, 'gini': 43.85747750957514}, 'col': 0, 'value': 0.32229999999999998, 'right': {'left': 1.0, 'col': 2, 'value': 

In [256]:
#递归测试
def run():
    f(1)
def f(i):
    print("enter digui:%d"%i)
    if(i < 3):
        f(i+1)
    print("exit digui:%d"%i)
run()

enter digui:1
enter digui:2
enter digui:3
exit digui:3
exit digui:2
exit digui:1
